# **Data Collection**

## Objectives

* Fetch data from Kaggle and prepare it for further processes.

## Inputs

* Kaggle JSON file - authentication token. 

## Outputs

* Generate Dataset: inputs/datasets/cherry_leaves_dataset.

## Additional Comments

* No comments. 



---

# Import packages & change the working directory

In [1]:
import numpy
import os


current_dir = os.getcwd()
current_dir

'/workspace/my-fifth-project/jupyter_notebooks'

In [2]:
os.chdir('/workspace/my-fifth-project')
print("You set a new current directory")

You set a new current directory


In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/my-fifth-project'

# Install Kagle

In [4]:
pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 29.9 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.12-py3-none-any.whl size=102972 sha256=efc5219bd1cb8dffeeec5c41ad8435fe07fe20a4b1fa8b517e4ea74d851b20b5
  Stored in directory: /workspace/.pyenv_mirror/pip_cache/wheels/a0/87/83/f9893bb1660a03ca5808b7429e77808120f091569651323dd1
Successfully built kaggle
Note: you may need to restart the kernel to use updated packages.


* Change the Kaggle configuration directory to the current working directory and set permissions for the Kaggle authentication JSON:

In [6]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

* Set the kaggle dataset and download it.

In [7]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherryleaves_database"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

Dataset URL: https://www.kaggle.com/datasets/codeinstitute/cherry-leaves
License(s): unknown
 93%|███████████████████████████████████▏  | 51.0M/55.0M [00:02<00:00, 28.6MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:02<00:00, 27.3MB/s]


* Unzip the downloaded file, delete the zip file.

In [8]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

---

# Data Preparation

### Data Cleaning

* Check and remove non image files.

In [11]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir) 
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        
        #print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location) # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file",len(j))
        print(f"Folder: {folder} - has non-image file",len(i))

* Summary of the remove_non_image_file() function.

In [16]:
remove_non_image_file(my_data_dir='inputs/cherryleaves_database/cherry-leaves')

Folder: healthy - has image file 2104
Folder: healthy - has non-image file 0
Folder: powdery_mildew - has image file 2104
Folder: powdery_mildew - has non-image file 0


### Split to train-validation-test set

In [17]:
import os
import shutil
import random
import joblib


def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):
    """
    Splits images into train, validation, and test sets and moves them to respective directories.

    Args:
    - my_data_dir: Path to the directory containing the original image folders.
    - train_set_ratio: Ratio of images to be allocated to the train set.
    - validation_set_ratio: Ratio of images to be allocated to the validation set.
    - test_set_ratio: Ratio of images to be allocated to the test set.
    """

    # Check if the sum of ratios equals 1.0
    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum 1.0")
        return

    # Get class labels from the original data directory
    labels = os.listdir(my_data_dir)

    # Create train, validation, test folders with class labels sub-folder
    if 'test' not in labels:
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        # Move images to respective sets
        for label in labels:
            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # Move file to train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)
                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # Move file to validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)
                else:
                    # Move file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)
                count += 1

            # Remove original label directory after moving files
            os.rmdir(my_data_dir + '/' + label)

Data distribution ratio: 
* Train set 0.7
* Validation set 0.1
* Test set 0.2

In [18]:
split_train_validation_test_images(my_data_dir = f"inputs/cherryleaves_database/cherry-leaves",
                        train_set_ratio = 0.7,
                        validation_set_ratio=0.1,
                        test_set_ratio=0.2
                        )

Summary of the new data sets.

In [19]:
def count_images_in_datasets(data_dir):
    """
    Counts the number of images in each dataset (train, validation, test) and for each class (healthy, powdery_mildew).
    """
    image_counts = {}

    # Loop through train, validation, and test directories
    for subset in ['train', 'validation', 'test']:
        subset_dir = os.path.join(data_dir, subset)
        image_counts[subset] = {}

        # Loop through class labels
        for label in os.listdir(subset_dir):
            label_dir = os.path.join(subset_dir, label)
            image_counts[subset][label] = 0
            
            # Count image files
            image_counts[subset][label] = len(os.listdir(label_dir))

    return image_counts

# Path to the directory containing the train, validation, and test datasets
data_directory = "inputs/cherryleaves_database/cherry-leaves"

# Count images in datasets
image_counts = count_images_in_datasets(data_directory)

# Display image counts
for subset, counts in image_counts.items():
    print(f"{subset.capitalize()} dataset:")
    for label, count in counts.items():
        print(f"  - {label}: {count} images.")

Train dataset:
  - healthy: 1472 images.
  - powdery_mildew: 1472 images.
Validation dataset:
  - healthy: 210 images.
  - powdery_mildew: 210 images.
Test dataset:
  - healthy: 422 images.
  - powdery_mildew: 422 images.


---